<a href="https://colab.research.google.com/github/ichhakumari/RAG-Model-for-QA-Bot/blob/main/RAG_Model_for_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Install libraries
!pip install faiss-cpu transformers sentence-transformers

In [12]:
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [13]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

dimension = 384
index = faiss.IndexFlatL2(dimension)

In [14]:
def add_documents_to_index(documents):
    embeddings = embedding_model.encode(documents)
    index.add(embeddings)
    return documents

documents = [
    "Our company provides IT consulting services.",
    "We specialize in artificial intelligence and machine learning.",
    "Our customer support is available 24/7.",
    "The pricing for our services starts at $500 per month."
]
document_texts = add_documents_to_index(documents)




In [15]:
def retrieve_relevant_documents(query, top_k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    return [document_texts[i] for i in indices[0]]



In [16]:
def qa_bot(query):
    relevant_docs = retrieve_relevant_documents(query)

    context = " ".join(relevant_docs)

    # a pre-trained language model
    qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-small")
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    response = qa_pipeline(prompt, max_length=100, truncation=True)
    return response[0]['generated_text']



In [17]:
# Test the bot
query = "What services does your company provide?"
response = qa_bot(query)
print(f"Question: {query}\nAnswer: {response}")

Device set to use cpu


Question: What services does your company provide?
Answer: IT consulting services


In [19]:
query = "What is price for your services ?"
response = qa_bot(query)
print(f"Question: {query}\nAnswer: {response}")

Device set to use cpu


Question: What is price for your services ?
Answer: The pricing for our services starts at $500 per month.
